In [39]:
# get the Dataset:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import scipy.io as sio

# load data from the .mat file:
# mat_contents = sio.loadmat('EEG_big.mat') # (EEG_data.mat')
# mat_contents = sio.loadmat('/home/amplifier/home/DATASETS/EEG_big4CNN.mat')
# mat_contents = sio.loadmat('/home/amplifier/home/DATASETS/EEG_big4CNN_1subj.mat')
# mat_contents = sio.loadmat('/home/amplifier/home/DATASETS/KOS_100Hz_ICA->0.5-38Hz,Env=1.mat')
# mat_contents = sio.loadmat('/home/amplifier/home/DATASETS/KOS_100Hz_noICA->0.5-38Hz,Env=1.mat')
mat_contents = sio.loadmat('/home/amplifier/home/DATASETS/Merged123->0.5-38Hz,Env=1.mat')

# mat_contents = sio.loadmat('/home/amplifier/home/DATASETS/Merged123->0.5-38Hz,Env=1.mat')

X = mat_contents['X']
Y = mat_contents['Z'].T

winsize = mat_contents['winsize']
stepsize = mat_contents['stepsize']
trial_len = mat_contents['trial_len']
low_cutoff = mat_contents['low_cutoff']
high_cutoff = mat_contents['high_cutoff']
source = mat_contents['filein']

# verify that the model REALLY finds a mapping between the input and the labels. If we get
# our accuracy by chance, then we should get the same accuracy on a permuted dataset:
# Y = np.random.permutation(Y)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
print('Original data type:', x_train.dtype)

# convert to float64 for numerical stability:
x_train = x_train.astype('float64')
y_train = y_train.astype('float64')
x_test = x_test.astype('float64')
y_test = y_test.astype('float64')

# normalize to unit variance and zero mean:
for i in range(x_train.shape[0]):
    x_train[i,:,:] = preprocessing.scale(x_train[i,:,:], axis=1)
    x_train[i,:,:] = preprocessing.minmax_scale(x_train[i,:,:], axis=1)

for i in range(x_test.shape[0]):
    x_test[i,:,:] = preprocessing.scale(x_test[i,:,:], axis=1)
    x_test[i,:,:] = preprocessing.minmax_scale(x_test[i,:,:], axis=1)

# ensure the tensors are sized right:
x_train = np.transpose(x_train,(0,2,1))
x_test = np.transpose(x_test,(0,2,1))

# one hot encode the labels:
onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
y_train = onehot_encoder.fit_transform(y_train)
y_test = onehot_encoder.fit_transform(y_test)

# convert to float16 to save space:
x_train = x_train.astype('float16')
y_train = y_train.astype('float16')
x_test = x_test.astype('float16')
y_test = y_test.astype('float16')
print('Normalized data type:', x_train.dtype)

leng = X.shape[2]

print('test input shape', x_test.shape, "Nomralized MEAN:", np.mean(x_test), "min", np.min(x_test),"max", np.max(x_test))
print('train input shape', x_train.shape, "Nomralized MEAN:", np.mean(x_train), "min", np.min(x_train),"max", np.max(x_train))

print('test labels shape', y_test.shape, "Nomralized MEAN:", np.mean(y_test), "min", np.min(y_test),"max", np.max(y_test))
print('train labels shape', y_train.shape, "Nomralized MEAN:", np.mean(y_train), "min", np.min(y_train),"max", np.max(y_train))

print('Window length', winsize)
print('Step size:', stepsize)
print('Length of trial during experiment', trial_len)
print('Low cutoff freq.', low_cutoff)
print('High cutoff freq.', high_cutoff)
print('Source data file:', source)

leng = x_test.shape[1]

Original data type: float32
Normalized data type: float16
test input shape (468, 400, 60) Nomralized MEAN: 0.51758 min 0.0 max 1.0
train input shape (1872, 400, 60) Nomralized MEAN: 0.5166 min 0.0 max 1.0
test labels shape (468, 2) Nomralized MEAN: 0.5 min 0.0 max 1.0
train labels shape (1872, 2) Nomralized MEAN: 0.5 min 0.0 max 1.0
Window length [[5]]
Step size: [[1]]
Length of trial during experiment [[30]]
Low cutoff freq. [[ 0.5]]
High cutoff freq. [[38]]
Source data file: ['Merged123']


In [40]:
# http://cs231n.github.io/convolutional-networks/#case
# we first define the autoencoder's architecture:

# make sure you don't hog all the video memory
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from keras import backend as K
K.set_session(sess)
###################################

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, Flatten, Conv1D, MaxPooling1D
from keras.models import Model
from keras.models import load_model
from keras.callbacks import TensorBoard, EarlyStopping, Callback
from keras import optimizers
from keras.initializers import Orthogonal as orth
import numpy as np
import matplotlib.pyplot as plt

enc = load_model('/home/amplifier/home/NEW_DL/models/EEGnet.h5')

enc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 400, 60)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 400, 60)           1600      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 400, 30)           5430      
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 30)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 400, 30)           120       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 200, 30)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 200, 60)           5460      
__________

In [41]:
# let's predict a TRAINING sammple:
decoded_train = enc.predict(x_train)
decoded_test = enc.predict(x_test)
print(decoded_train.shape)
print(decoded_test.shape)

(1872, 2)
(468, 2)


In [42]:
# now test sample by sample
cor_test = []
L = x_test.shape
for sample_no in range(x_test.shape[0]):
    pred = enc.predict(np.expand_dims(x_test[sample_no],0))
    if ((pred[0][0]<pred[0][1]) and (y_test[sample_no,0] < y_test[sample_no,1]) or
       ((pred[0][0]>pred[0][1]) & (y_test[sample_no,0] > y_test[sample_no,1]))):
        cor_test.append(1)
    else:
        cor_test.append(0)

cor_train = []
L = x_train.shape
for sample_no in range(x_train.shape[0]):
    pred = enc.predict(np.expand_dims(x_train[sample_no],0))
    if ((pred[0][0]<pred[0][1]) and (y_train[sample_no,0] < y_train[sample_no,1]) or
       ((pred[0][0]>pred[0][1]) & (y_train[sample_no,0] > y_train[sample_no,1]))):
        cor_train.append(1)
    else:
        cor_train.append(0)

# Report accuracies
print('Accuracy on the test data:', np.mean(cor_test))
print('Accuracy on the training data:', np.mean(cor_train))

Accuracy on the test data: 0.967948717949
Accuracy on the training data: 0.967948717949
